In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from numpy.random import default_rng
rng = default_rng()

In [ ]:
def call(T, S0, K, sigma, r, q):
    d1 = np.log(S0*np.exp((r-q)*T)/K)/(sigma*np.sqrt(T)) + 0.5*sigma*np.sqrt(T)
    d2 = d1 - sigma*np.sqrt(T)
    return S0*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def delta(T, S0, K, sigma, r, q):
    d1 = np.log(S0*np.exp((r-q)*T)/K)/(sigma*np.sqrt(T)) + 0.5*sigma*np.sqrt(T)
    return np.exp(-q*T)*norm.cdf(d1)

def gamma(T, S0, K, sigma, r, q):
    d1 = np.log(S0*np.exp((r-q)*T)/K)/(sigma*np.sqrt(T)) + 0.5*sigma*np.sqrt(T)
    return np.exp(-q*T)*norm.pdf(d1)/(S0*sigma*np.sqrt(T))

def deltaF0(T, F0, K, sigma, q=0):
    d1 = np.log(F0*np.exp((-q)*T)/K)/(sigma*np.sqrt(T)) + 0.5*sigma*np.sqrt(T)
    return np.exp(-q*T)*norm.cdf(d1)

def delta_putF0(T, F0, K, sigma, q):
    d1 = np.log(F0*np.exp((-q)*T)/K)/(sigma*np.sqrt(T)) + 0.5*sigma*np.sqrt(T)
    return np.exp(-q*T)*(norm.cdf(d1)-1)

 # Profit and Loss (PnL) at maturity of a Delta hedging strategy

In [ ]:
file = open('téléchargement (17).csv')
for i, line in zip(range(5), file):
    print(line.strip())

Date;Spot;
0.000000;94.010000;
0.004000;93.840000;
0.008000;94.470000;
0.012000;92.620000;


In [ ]:
df = pd.read_csv('téléchargement (17).csv', delimiter=";").drop(columns='Unnamed: 2')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    231 non-null    float64
 1   Spot    231 non-null    float64
dtypes: float64(2)
memory usage: 3.7 KB


In [ ]:
T = 0.266
K = 103.3
r = 0.002
sigma = 0.2
q = 0

S = df["Spot"].to_numpy()
temps = df["Date"].to_numpy()
dt = np.diff(temps)

max(S[-1]-K,0)

47.269999999999996

In [ ]:
V = np.empty(len(S))
V[0] = call(T=T, S0=S[0], K=K, sigma=sigma, r=r, q=q)
deltas = delta(T=T-temps[:-1], S0=S[:-1], K=K, sigma=sigma, r=r, q=q)
deltas[1:] = deltas[:-1] + np.diff(deltas)
#deltas[i] * S[i] * (1 - np.exp(r*dt[i]))

for i in range(len(S)-1):
    V[i+1] = deltas[i] * S[i+1] + (V[i]- deltas[i]*S[i]) * np.exp(r*dt[i])

<ipython-input-2-88748bafa6f7>:7: RuntimeWarning: invalid value encountered in sqrt
  d1 = np.log(S0*np.exp((r-q)*T)/K)/(sigma*np.sqrt(T)) + 0.5*sigma*np.sqrt(T)


In [ ]:
len(S)

214

In [ ]:
len(deltas)

213

In [ ]:
V[-1] - max(S[-1]-K,0)

nan

# Robustness of the Black and Scholes Formula

In [ ]:
file = open('téléchargement (17).csv')
for i, line in zip(range(5), file):
    print(line.strip())

Date;Spot;
0.000000;94.010000;
0.004000;93.840000;
0.008000;94.470000;
0.012000;92.620000;


In [ ]:
df = pd.read_csv('téléchargement (17).csv', delimiter=";").drop(columns='Unnamed: 2')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    231 non-null    float64
 1   Spot    231 non-null    float64
dtypes: float64(2)
memory usage: 3.7 KB


In [ ]:
K = 96.55
T = 0.920
r = 4.9/100
q = 0
vol_mod = 22.7/100
vol_real = 20.5/100
dvol = vol_mod**2 - vol_real**2

S = df["Spot"].to_numpy()
temps = df["Date"].to_numpy()

dt = np.diff(temps)
gammas = gamma(T=T-temps[:-1], S0=S[:-1], K=K, sigma=vol_mod, r=r, q=q)
intégrande = np.exp(-r*temps[:-1]) * S[:-1]**2 * gammas * dvol

PL = 0.5* np.exp(r*T) * (dt * intégrande).sum()
PL

0.6149642287627773

# Implied Volatility smile

In [ ]:
file = open('data/Implied-Volatility5.csv')
for i, line in zip(range(5), file):
    print(line.strip())

Strike;IV;
25.393322;0.867020;
25.464595;0.866415;
25.536067;0.865810;
25.607740;0.865205;


In [ ]:
df = pd.read_csv('data/Implied-Volatility5.csv', delimiter=";")
df.info()
#df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Strike      1000 non-null   float64
 1   IV          1000 non-null   float64
 2   Unnamed: 2  0 non-null      float64
dtypes: float64(3)
memory usage: 23.6 KB


In [ ]:
T = 0.593
F0 = 102.975

strk = df["Strike"].to_numpy()
iv = df["IV"].to_numpy()

In [ ]:
delt = deltaF0(T=T, F0=F0, K=strk, sigma=iv, q=0)

cond_25 = (delt<0.2516) & (delt>0.247)
cond_50 = (delt<0.5031) & (delt>0.497)
cond_75 = (delt<0.7521) & (delt>0.749)

print(delt[cond_25])
print(delt[cond_50])
print(delt[cond_75])

[0.25035405 0.24783395]
[0.50240566 0.49943816]
[0.75115667 0.74924942]


In [ ]:
x = delt[cond_25]
y = iv[cond_25]
#print(y)

a = (y[1]-y[0])/(x[1]-x[0])
b = y[0]-a*x[0]
ivcall_25 = a*0.25+b

print("IV for strike at which the Call Delta equal 25% : ", ivcall_25)
#plt.plot(x,y)
#plt.show()

IV for strike at which the Call Delta equal 25% :  0.48457890170731405


In [ ]:
x = delt[cond_50]
y = iv[cond_50]
#print(y)

a = (y[1]-y[0])/(x[1]-x[0])
b = y[0]-a*x[0]
ivcall_50 = a*0.5+b

print("IV for strike at which the Call Delta equal 50% : ", ivcall_50)
#plt.plot(x,y)
#plt.show()

IV for strike at which the Call Delta equal 50% :  0.5173211555658743


In [ ]:
x = delt[cond_75]
y = iv[cond_75]
#print(y)

a = (y[1]-y[0])/(x[1]-x[0])
b = y[0]-a*x[0]
ivcall_75 = a*0.75+b

print("IV for strike at which the Call Delta equal 75% : ", ivcall_75)
#plt.plot(x,y)
#plt.show()

IV for strike at which the Call Delta equal 75% :  0.5799513609897642


In [ ]:
print("Vol risk metrics for butterfly position : ", 0.5*(ivcall_25 + ivcall_75) - ivcall_50 )
print("Vol risk metrics for risk-reversal position : ", ivcall_25 - ivcall_75 )

Vol risk metrics for butterfly position :  0.01494397578266482
Vol risk metrics for risk-reversal position :  -0.09537245928245014


In [ ]:
delt = delta_putF0(T=T, F0=F0, K=strk, sigma=iv, q=0)
cond = (delt>-0.753) & (delt<-0.749)
print(delt[cond])

[-0.74964595 -0.75216605]


In [ ]:
x = delt[cond]
y = iv[cond]
#print(y)

a = (y[1]-y[0])/(x[1]-x[0])
b = y[0]-a*x[0]
ivput_75 = -a*0.75+b

print(ivput_75)
#plt.plot(x,y)
#plt.show()

0.48457890170731405


In [ ]:
print(ivcall_25 == ivput_75)

True


# VWAP

In [ ]:
file = open('data/vwap3.csv')
for i, line in zip(range(5), file):
    print(line.strip())

Price;Volume;
294.518930;0.226657;
295.175774;0.163936;
294.043418;0.175429;
295.518301;0.128019;


In [ ]:
df = pd.read_csv('data/vwap3.csv', delimiter=";")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Price       158 non-null    float64
 1   Volume      158 non-null    float64
 2   Unnamed: 2  0 non-null      float64
dtypes: float64(3)
memory usage: 3.8 KB


In [ ]:
price= df["Price"].to_numpy()
volume = df["Volume"].to_numpy()

(price*volume).sum() / volume.sum()

293.43787785365214

# VWM

In [ ]:
file = open('data/vwm3.csv')
for i, line in zip(range(5), file):
    print(line.strip())

Price;Volume;
150.173980;0.749963;
148.717362;0.122394;
149.928079;0.100547;
148.876215;0.154720;


In [ ]:
df = pd.read_csv('data/vwm3.csv', delimiter=";")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Price       131 non-null    float64
 1   Volume      131 non-null    float64
 2   Unnamed: 2  0 non-null      float64
dtypes: float64(3)
memory usage: 3.2 KB


In [ ]:
price = df["Price"].to_numpy()
volume = df["Volume"].to_numpy()
order = np.argsort(price)
sorted_p = price[order]
sorted_v = volume[order]
cum_v = sorted_v.cumsum()
sum_v = volume.sum()
index = np.flatnonzero(cum_v/sum_v >= 0.5)[0]
print((cum_v/sum_v)[index-1], (cum_v/sum_v)[index])
print(sorted_p[index-1], sorted_p[index])
weight = ( 0.5 - (cum_v/sum_v)[index-1] ) / ( (cum_v/sum_v)[index] - (cum_v/sum_v)[index-1] )
weight

0.4980325041514574 0.5067950227472652
149.436697 149.438993


0.22453542631954138

In [ ]:
(1-weight)*sorted_p[index-1] + weight*sorted_p[index]

149.43721253333882

In [ ]:
T = 5
Y = np.linspace(0,T,100)
T*Y.mean()

12.5

In [ ]:
(np.diff(Y)*(Y[:-1]+Y[1:])/2).sum()

12.499999999999998